In [17]:
import os
import time
from concurrent import futures
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import pandas as pd

In [3]:
assert load_dotenv(find_dotenv())

In [4]:
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

In [12]:
colors = ['aliceblue', 'antiquewhite', 'aqua', 'aquamarine', 'azure', 'beige', 'bisque', 'black',
          'blanchedalmond', 'blue', 'blueviolet', 'brown', 'burlywood', 'cadetblue', 'chartreuse',
          'chocolate', 'coral', 'cornflowerblue', 'cornsilk', 'crimson', 'cyan', 'darkblue',
          'darkcyan', 'darkgoldenrod', 'darkgray', 'darkgrey', 'darkgreen', 'darkkhaki', 'darkmagenta',
          'darkolivegreen', 'darkorange', 'darkorchid', 'darkred', 'darksalmon', 'darkseagreen',
          'darkslateblue', 'darkslategray', 'darkslategrey', 'darkturquoise', 'darkviolet',
          'deeppink', 'deepskyblue', 'dimgray', 'dimgrey', 'dodgerblue', 'firebrick',
          'floralwhite', 'forestgreen', 'fuchsia', 'gainsboro', 'ghostwhite', 'gold', 'goldenrod',
          'gray', 'grey', 'green', 'greenyellow', 'honeydew', 'hotpink', 'indianred', 'indigo']

### Single request - example

In [21]:
def completion_single(placeholder):
    # Change the messages as needed:
    messages=[
        {"role": "system", "content": "You are an expert in web design."},
        {"role": "user", "content": "give me the hex color code of the color: placeholder"},
    ]
    for message in messages:
        message['content'] = message['content'].replace("placeholder", placeholder)
    resp = client.chat.completions.create(
        messages=messages,
        # Optional: modify model and add any other parameters
        model="gpt-4o-2024-08-06",
    )
    # return resp.choices[0].message.content
    return resp

In [22]:
resp = completion_single(placeholder='green')

In [23]:
from rich.pretty import pprint
pprint(resp)

ChatCompletion(
│   id='chatcmpl-ApcNTxuqwPGR8HtWC1AijXS3cj6gJ',
│   choices=[
│   │   Choice(
│   │   │   finish_reason='stop',
│   │   │   index=0,
│   │   │   logprobs=None,
│   │   │   message=ChatCompletionMessage(
│   │   │   │   content='The hex color code for the color green is #008000.',
│   │   │   │   refusal=None,
│   │   │   │   role='assistant',
│   │   │   │   audio=None,
│   │   │   │   function_call=None,
│   │   │   │   tool_calls=None
│   │   │   )
│   │   )
│   ],
│   created=1736865615,
│   model='gpt-4o-2024-08-06',
│   object='chat.completion',
│   service_tier='default',
│   system_fingerprint='fp_50cad350e4',
│   usage=CompletionUsage(
│   │   completion_tokens=14,
│   │   prompt_tokens=30,
│   │   total_tokens=44,
│   │   completion_tokens_details=CompletionTokensDetails(
│   │   │   accepted_prediction_tokens=0,
│   │   │   audio_tokens=0,
│   │   │   reasoning_tokens=0,
│   │   │   rejected_prediction_tokens=0
│   │   ),
│   │   prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)
│   )
)

In [26]:
resp.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The hex color code for the color green is #008000.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))

### thread

In [28]:
def completion_multi(placeholders):
    final_result = []
    total = len(placeholders)
    completed = 0
    
    print(f"开始处理 {total} 个任务...")
    
    with futures.ThreadPoolExecutor(max_workers=8) as executor:
        future_to_placeholder = {}
        for placeholder in placeholders:
            future = executor.submit(completion_single, placeholder)
            future_to_placeholder[future] = placeholder
            
        for future in futures.as_completed(future_to_placeholder):
            placeholder = future_to_placeholder[future]
            result = future.result()
            final_result.append((placeholder, result))
            
            # 更新并显示进度
            completed += 1
            progress = (completed / total) * 100
            print(f"进度: {completed}/{total} ({progress:.1f}%)", end='\r')
    
    print("\n所有任务处理完成！")
    return final_result

In [29]:
results = completion_multi(colors)

开始处理 61 个任务...
进度: 61/61 (100.0%)
所有任务处理完成！


In [30]:
results

[('antiquewhite',
  ChatCompletion(id='chatcmpl-ApcS0YTSgVAJHteb443MHgYzbA0kp', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The hex color code for "AntiqueWhite" is #FAEBD7.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1736865896, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_4691090a87', usage=CompletionUsage(completion_tokens=18, prompt_tokens=33, total_tokens=51, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))),
 ('aliceblue',
  ChatCompletion(id='chatcmpl-ApcS00eUoA7JpSg7pnJ2VMuuvyJw4', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The hex color code for AliceBlue is #F0F8FF.', refusal=None, role='